In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

In [2]:
df = pd.read_csv("spotify_reviews.csv")

In [3]:
df.shape

(40000, 6)

In [4]:
df.columns

Index(['Review', 'Rating', 'sentiment', 'review_after_removing_html_tags',
       'review_after_removing_punctuation',
       'review_after_pos_and_lemmatization'],
      dtype='object')

In [5]:
df.isna().sum()

Review                                0
Rating                                0
sentiment                             0
review_after_removing_html_tags       0
review_after_removing_punctuation     0
review_after_pos_and_lemmatization    0
dtype: int64

In [6]:
df[ df.sentiment == 1 ].shape

(20000, 6)

In [7]:
df[ df.sentiment == 0 ].shape

(20000, 6)

In [11]:
bag_of_word_model = CountVectorizer(analyzer='word', ngram_range=(1,3))

X = bag_of_word_model.fit_transform(df["review_after_pos_and_lemmatization"])
y = df["sentiment"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
y_train[ y_train == 1 ].shape

(16036,)

In [14]:
y_train[ y_train == 0 ].shape

(15964,)

In [15]:
y_test[ y_test == 1 ].shape

(3964,)

In [16]:
y_test[ y_test == 0 ].shape

(4036,)

#### KFold cross-validation

In [17]:
# Set up k-fold cross-validation
kf = KFold(n_splits=4, shuffle=True)

random_forest_scores = []
logistic_scores = []
svm_scores = []
gd_boost_scores = []

# Generate indices for each fold
for train_index, test_index in kf.split(X):
    x_train_data, x_test_data = X[train_index], X[test_index]
    y_train_data, y_test_data = y[train_index], y[test_index]

    ## Random Forest
    model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

    print("Fitting random forest to training data....")    
    model_forest = model_forest.fit(x_train_data, y_train_data)
    
    y_pred_data = model_forest.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    random_forest_scores.append(score)
    print(random_forest_scores)
    
    ## Logistic Regression
    
    model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
    print("Fitting Logistic Regression to training data....")  
    model_logistic_regression = model_logistic_regression.fit(x_train_data, y_train_data)
    
    y_pred_data = model_logistic_regression.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    logistic_scores.append(score)
    print(logistic_scores)
    
    ## SVM
    model_svm = SVC(C=0.1, kernel='linear')
    print("Fitting SVM to training data....")  
    model_svm = model_svm.fit(x_train_data, y_train_data)
    
    y_pred_data = model_svm.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    svm_scores.append(score)
    print(svm_scores)
    
    ## Gradient Boosting
    gb_classifier = GradientBoostingClassifier(random_state=2)
    print("Fitting Gradient Boosting to training data....")    
    gb_classifier.fit(x_train_data, y_train_data)
    
    y_pred_data = gb_classifier.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    gd_boost_scores.append(score)
    print(gd_boost_scores)
    
    print("=======================================================")

# print(f"Cross-validation scores: {scores}")
# print(f"Average score: {np.mean(scores)}")

Fitting random forest to training data....
[0.7992]
Fitting Logistic Regression to training data....
[0.8417]
Fitting SVM to training data....
[0.8446]
Fitting Gradient Boosting to training data....
[0.811]
Fitting random forest to training data....
[0.7992, 0.7796]
Fitting Logistic Regression to training data....
[0.8417, 0.8402]
Fitting SVM to training data....
[0.8446, 0.847]
Fitting Gradient Boosting to training data....
[0.811, 0.806]
Fitting random forest to training data....
[0.7992, 0.7796, 0.819]
Fitting Logistic Regression to training data....
[0.8417, 0.8402, 0.845]
Fitting SVM to training data....
[0.8446, 0.847, 0.8504]
Fitting Gradient Boosting to training data....
[0.811, 0.806, 0.8116]
Fitting random forest to training data....
[0.7992, 0.7796, 0.819, 0.7782]
Fitting Logistic Regression to training data....
[0.8417, 0.8402, 0.845, 0.8437]
Fitting SVM to training data....
[0.8446, 0.847, 0.8504, 0.8497]
Fitting Gradient Boosting to training data....
[0.811, 0.806, 0.8116

In [18]:
print("Random Forest KFold cross-validation")
print(f"Cross-validation scores: {random_forest_scores}")
print(f"Average score: {np.mean(random_forest_scores)}\n")
print("Logistic Regression KFold cross-validation")
print(f"Cross-validation scores: {logistic_scores}")
print(f"Average score: {np.mean(logistic_scores)}\n")
print("SVM cross-validation")
print(f"Cross-validation scores: {svm_scores}")
print(f"Average score: {np.mean(svm_scores)}\n")
print("Gradient Boosting KFold cross-validation")
print(f"Cross-validation scores: {gd_boost_scores}")
print(f"Average score: {np.mean(gd_boost_scores)}\n")

Random Forest KFold cross-validation
Cross-validation scores: [0.7992, 0.7796, 0.819, 0.7782]
Average score: 0.794

Logistic Regression KFold cross-validation
Cross-validation scores: [0.8417, 0.8402, 0.845, 0.8437]
Average score: 0.84265

SVM cross-validation
Cross-validation scores: [0.8446, 0.847, 0.8504, 0.8497]
Average score: 0.8479249999999999

Gradient Boosting KFold cross-validation
Cross-validation scores: [0.811, 0.806, 0.8116, 0.8097]
Average score: 0.8095749999999999



#### Random Forest

In [19]:
# Start the timer
start_time = datetime.now()

model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

print("Fitting random forest to training data....")    
model_forest = model_forest.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Training Runtime:', runtime)

Fitting random forest to training data....
Random Forest Training Runtime: 0:01:58.723969


In [20]:
model_forest.score(X_test, y_test)

0.790125

In [21]:
model_forest.score(X_train, y_train)

0.80440625

In [22]:
# Start the timer
start_time = datetime.now()

y_pred = model_forest.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Prediction Runtime:', runtime)

Random Forest Prediction Runtime: 0:00:02.802916


In [23]:
y_pred = cross_val_predict(model_forest, X_test, y_test)

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81      4036
           1       0.81      0.80      0.81      3964

    accuracy                           0.81      8000
   macro avg       0.81      0.81      0.81      8000
weighted avg       0.81      0.81      0.81      8000



In [25]:
print(confusion_matrix(y_test, y_pred))

[[3309  727]
 [ 790 3174]]


In [26]:
filename = 'models/rd_ngram_ml_model.sav'
pickle.dump(model_forest, open(filename, 'wb'))

#### Logistic Regression

In [27]:
# Start the timer
start_time = datetime.now()

model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

print("Fitting Logistic Regression to training data....")  
model_logistic_regression = model_logistic_regression.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Training Runtime:', runtime)

Fitting Logistic Regression to training data....
Logistic Regression Training Runtime: 0:00:01.287388


In [28]:
model_logistic_regression.score(X_test, y_test)

0.841875

In [29]:
model_logistic_regression.score(X_train, y_train)

0.85078125

In [30]:
# Start the timer
start_time = datetime.now()

y_pred = model_logistic_regression.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Prediction Runtime:', runtime)

Logistic Regression Prediction Runtime: 0:00:00


In [31]:
y_pred = cross_val_predict(model_logistic_regression, X_test, y_test)

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82      4036
           1       0.82      0.80      0.81      3964

    accuracy                           0.81      8000
   macro avg       0.81      0.81      0.81      8000
weighted avg       0.81      0.81      0.81      8000



In [33]:
print(confusion_matrix(y_test, y_pred))

[[3361  675]
 [ 811 3153]]


In [34]:
filename = 'models/lg_ngram_ml_model.sav'
pickle.dump(model_logistic_regression, open(filename, 'wb'))

#### SVM

In [35]:
# Start the timer
start_time = datetime.now()

model_svm = SVC(C=0.1, kernel='linear')

model_svm = model_svm.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:12:56.962399


In [36]:
model_svm.score(X_test, y_test)

0.8495

In [37]:
model_svm.score(X_train, y_train)

0.99209375

In [38]:
# Start the timer
start_time = datetime.now()

y_pred = model_svm.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:31.860677


In [39]:
y_pred = cross_val_predict(model_svm, X_test, y_test)

In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      4036
           1       0.82      0.84      0.83      3964

    accuracy                           0.83      8000
   macro avg       0.83      0.83      0.83      8000
weighted avg       0.83      0.83      0.83      8000



In [41]:
print(confusion_matrix(y_test, y_pred))

[[3319  717]
 [ 639 3325]]


In [42]:
filename = 'models/svm_ngram_ml_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))

#### Gradient Boosting Classifier

In [43]:
# Start the timer
start_time = datetime.now()


# create a Gradient Boosting classifier with default hyperparameters
gb_classifier = GradientBoostingClassifier(random_state=2)

# fit the classifier to the training data
gb_classifier.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Gradient Boosting Prediction Runtime:', runtime)


Gradient Boosting Prediction Runtime: 0:47:38.440453


In [44]:
gb_classifier.score(X_test, y_test)

0.805875

In [45]:
gb_classifier.score(X_train, y_train)

0.8151875

In [46]:
# Start the timer
start_time = datetime.now()

y_pred = gb_classifier.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:00.015446


In [47]:
y_pred = cross_val_predict(gb_classifier, X_test, y_test)

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81      4036
           1       0.81      0.80      0.80      3964

    accuracy                           0.81      8000
   macro avg       0.81      0.81      0.81      8000
weighted avg       0.81      0.81      0.81      8000



In [49]:
filename = 'models/gb_boosting_ngram_ml_model.sav'
pickle.dump(gb_classifier, open(filename, 'wb'))